In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Dec 18 09:37:48 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
%cd /content/
!rm -rf /content/fastpsf

import urllib.request
import os
from IPython.utils import io

if not os.path.exists("/content/fastpsf.zip"):
  fastpsf_colab_binary = "https://www.dropbox.com/scl/fi/hxue3cdi9vghkhnroh1c8/fastpsf-colab-binary.zip?rlkey=w5bvn0qxqhqtyv8t4r6ka1fyf&dl=1"
  urllib.request.urlretrieve(fastpsf_colab_binary, "/content/fastpsf.zip")

if not os.path.exists("/content/fastpsf"):
  print("Unzipping SMLM PSF fitter...")

  with io.capture_output() as captured:
    !unzip /content/fastpsf.zip

if False: # alternatively, compile from source
  %cd /content
  !git clone https://gitlab.com/jcnossen/fastpsf.git
  %cd /content/fastpsf
  !cmake .
  !make clean & make


/content
Unzipping SMLM PSF fitter...


In [ ]:
%cd /content/fastpsf/python
%pip install -e .
# Google colab now requires either this, or restarting the runtime after doing a local package install
import site
site.main()


In [ ]:
# A quick check to see that CUDA is working ok
from fastpsf import Context, GaussianPSFMethods

with Context() as ctx:
    m = GaussianPSFMethods(ctx)

    psf = m.CreatePSF_XYIBg(10, 1.3, cuda=True)
    smp = psf.ExpectedValue([[4,5,500,2]])
    assert smp.sum()>0


In [ ]:
!rm -rf /content/simcode_model.zip
!rm -rf /content/simcode_model

import urllib.request
simcode_model_url = "https://www.dropbox.com/scl/fi/r96qdxgfmykz7e2pk87dh/sf_conv_g1.3_tio2_L64_2.zip?rlkey=8wjtfe77qv7wnpadilg6p2qbq&st=9un2rgr1&dl=1"
urllib.request.urlretrieve(simcode_model_url, "/content/simcode_model.zip")
!unzip /content/simcode_model.zip -d /content/simcode_model

In [ ]:
%cd /content
!git clone https://github.com/jcnossen/sc.git
%cd /content/sc/
!pip install -e .

!cp /content/sc/example/generate_tubules.py /content/
%cd /content
# Google colab now requires either this, or restarting the runtime after doing a local package install
import site
site.main()


In [ ]:
%cd /content/sc/
!git pull

In [ ]:
import smlmtorch.simflux.pattern_estimator as pe
import numpy as np
import matplotlib.pyplot as plt
from smlmtorch.util.multipart_tiff  import MultipartTiffSaver, tiff_read_all
from smlmtorch import Dataset
from smlmtorch.simflux.simulate import simulate, angles_to_mod
from smlmtorch.simflux import SFLocalizer
import tqdm
from smlmtorch.simflux.localizer_report import LocalizationReporter


In [ ]:
from generate_tubules import generate_microtubule_points

import matplotlib.pyplot as plt
import numpy as np
from IPython.utils import io

np.random.seed(2)
W = 50
pixelsize = 100 #nm

with io.capture_output() as captured: # suppress outputs
  pts = generate_microtubule_points(W, depth=0, numtubules = 20, spl_knots=5, plot=False,
                  linedensity=10, nudge_factor=0.1, margin=0.05, straight_ish=True)

  gt_ds = Dataset(len(pts), 2, [W,W], pixelsize=pixelsize)
  gt_ds.pos = pts[:,:2]
  gt_ds.save('/content/ground-truth-points.hdf5')

plt.figure();
plt.scatter(pts[:,0],pts[:,1],s=1);
# invert bottom/top to match imshow
plt.gca().invert_yaxis();
plt.xlabel('Pixels'); plt.ylabel('Pixels');
plt.title("Simulated microtubules - ground truth")


In [ ]:
# Simulate an SMLM recording with modulated illumination
on_fraction = 0.01
avg_on_time = 6
nframes = 10000
intensity = 500
psf_sigma = 1.3
psf_calib = [psf_sigma, psf_sigma] # XY

background = 4 # approximately dna paint background on our dmd setup
path = '/content/movie.tif'

pixelsize = 100
pattern_frames = np.array([  # Defines which frame indices have which modulation pattern
    [0,1,2], # X pattern frame indices
    [3,4,5]  # Y pattern frame indices
])
modulation_depth = 0.90
modulation_angles_deg = [0, 90]
pitch_nm = 220
mod = angles_to_mod([pitch_nm, pitch_nm], pixelsize, modulation_angles_deg, modulation_depth, pattern_frames)
mp_gt = pe.ModulationPattern(pattern_frames, mod)

with Context() as ctx:
  roisize = 10
  psf = GaussianPSFMethods(ctx).CreatePSF_XYIBg(roisize, psf_calib, cuda=True)
  mov = simulate(path, mp_gt, psf,
      pts[:,:2], numframes=nframes, intensity=intensity, width=W,
      bg=background, avg_on_time=avg_on_time, on_fraction=on_fraction, return_movie=True)



In [ ]:
# Show the first few frames:
frames = tiff_read_all(path, maxframes=6)
fig,ax=plt.subplots(1,len(frames),figsize=(15,5))
for i in range(len(frames)):
    ax[i].imshow(frames[i])
    ax[i].axis('off')
    ax[i].set_aspect('equal')
    ax[i].set_title(f'frame {i}')
plt.tight_layout()

In [ ]:
plt.imshow(mov.mean(0));

In [ ]:
# For comparison, we detect spots in a single emitter pipeline first, and run both SMLM and SIMFLUX fitters on the found ROIs

localizer = SFLocalizer(path,
   psf_calib = [psf_sigma, psf_sigma],
   roisize = roisize,
   detection_threshold = 5,
   pattern_frames= pattern_frames,
   gain = 1,
   offset = 0,
   pixelsize = pixelsize,
   zrange = [0,0], # unsupported
   psf_sigma_binsize = None, # if non-zero, it will estimate the PSF width over time with given bin size
   result_dir='results',
   device='cuda:0')

localizer.detect_spots(ignore_cache=False, moving_window=True)
smlm_ds = localizer.fit_smlm(max_crlb_xy=None, ignore_cache=False)
print(f"numrois: {localizer.numrois}. #summed_fits: {localizer.summed_fits.shape[0]}, numframes: {smlm_ds.numFrames}")


In [ ]:
fig,ax = plt.subplots(figsize=(8,8))
smlm_ds.renderFigure(axes=ax,zoom=10,clip_percentile=98)

In [ ]:
# Estimate patterns so we can run simflux fitting.
# Note that in our method, optical setup modulation depths are highly underestimated in high density scenes,
# so in practice we fix based on our estimates from low density measurements
mp_est = localizer.estimate_angles(pitch_minmax_nm=[100,240])
mp_est = localizer.estimate_phases(mp_est, spots_per_bin=5000,
                                accept_percentile=40, iterations=10, verbose=False)
mp_est.depths[:]=0.9
# we also assume constant phase steps over time
mp_est = mp_est.const_phase_offsets()


In [ ]:
# filter based on modulation error:
# we calculate the expected intensities based on the SMLM position and our patterns, if too different then they are likely unreliable for a pattern based fit
me = mp_est.mod_error(smlm_ds)
me_sel = me < 0.1

sf_ds = localizer.fit_simflux(mp_est, smlm_ds[me_sel], iterations=50, lambda_=500, ignore_cache=True, normalizeWeights=True, distFromSMLM=0.5)
#lr.scatterplot([ sfloc.sum_ds, sf_ds ], connected=False, labels=['SMLM', 'SF'], limits=None, s=2)

if False: #show intermediate results
  fig,ax=plt.subplots(1,3,figsize=(15,5))
  gt_ds.renderFigure(axes=ax[0],clip_percentile=90,zoom=10,title='Ground truth')
  smlm_ds.renderFigure(axes=ax[1],zoom=10,clip_percentile=98,title='SMLM')
  sf_ds.renderFigure(axes=ax[2],clip_percentile=98,zoom=10,title='SIMFLUX')


In [ ]:
# Now run the simcode pipeline
from smlmtorch.util.config_dict import config_dict # dictionary indexable like object
from smlmtorch.nn.sf_model import SFLocalizationModel
from smlmtorch.nn.localize_movie import MovieProcessor
from smlmtorch.simflux.dataset import SFDataset
import torch

simcode_model_path = '/content/simcode_model/sf_conv_g1.3_tio2_L64_2'
simcode_model_class = SFLocalizationModel

config = config_dict.load(simcode_model_path + '/config.yaml')
config = config_dict(**config, # Need to add some inference related parameters
  detector=dict( prob_threshold=0.7, use_prob_weights=False )
)
config.model.num_intensities = pattern_frames.size

mp = MovieProcessor(simcode_model_class, config, simcode_model_path+"/checkpoint_1.pt", device='cuda:0')
simcode_nn_ds = mp.process(path, batch_size=32, batch_overlap=pattern_frames.size-1, gain=1, offset=0)
simcode_nn_ds.crlb_filter(0.3) # reduce the size a bit, it generates a lot of low precision spots
simcode_nn_ds['pixelsize'] = pixelsize

# Saving in .npy will keep track of per-pattern spot intensities.
simcode_nn_ds.save('/content/results/simcode-nn.npy')
# Saving in hdf5 is Picasso compatible, but will lose per-pattern spot intensities,
# and only store a single intensity per localization
simcode_nn_ds.save('/content/results/simcode-nn.hdf5')


In [ ]:
# Run pattern fits on NN outputs
mp_sc = pe.estimate_angles(pitch_minmax_nm=[150,500], ds=simcode_nn_ds,
                        pattern_frames = pattern_frames, result_dir = localizer.result_dir,
                        device = localizer.device, moving_window = localizer.moving_window)

mp_sc = pe.estimate_phases(simcode_nn_ds, mp_sc, spots_per_bin=10000,
                        accept_percentile=50, iterations=1, verbose=False, device=localizer.device)
mod_pattern_nn = mp_sc.const_phase_offsets()
# Plot phases
fig,ax=plt.subplots(2,1,figsize=(6,4))
mp_sc.plot_phase_drift(nframes=10000,ax=ax, label='Pat {0}', linestyle='-', lw=1)


In [ ]:
mod_error_threshold = 0.1
print(f'computing modulation-enhanced positions using mod error threshold = {mod_error_threshold}')
moderr_selected = mod_pattern_nn.mod_error(simcode_nn_ds) < mod_error_threshold

ndi_input_ds = simcode_nn_ds[moderr_selected]
simcode_pattern_fit_ds = pe.ndi_fit_dataset(ndi_input_ds, mod_pattern_nn, device=localizer.device)

# Filter outliers based on the distance from the non pattern-fitted positions
max_dist = 0.2
dist = np.sqrt ( ( (simcode_pattern_fit_ds.pos[:,:2] - ndi_input_ds.pos[:,:2])**2 ).sum(1) )
simcode_pattern_fit_ds = simcode_pattern_fit_ds[dist<max_dist]
simcode_pattern_fit_ds.save(localizer.result_dir + "simcode-pattern-fitted.hdf5")

print(f'remaining after filtering by max distance from original: {len(simcode_pattern_fit_ds)}/{moderr_selected.sum()}')

simcode_merged_ds = pe.merge_estimates(simcode_pattern_fit_ds, ndi_input_ds[dist<max_dist])
simcode_merged_ds.save("/content/results/simcode.hdf5")




In [ ]:
fig,ax=plt.subplots(1,2,figsize=(10,5))
border = 10
simcode_nn_ds.cropFromEdge(border).renderFigure(axes=ax[0],clip_percentile=95,zoom=10,title='SIMCODE (Intermediate NN output)')
simcode_merged_ds.cropFromEdge(border).renderFigure(axes=ax[1],clip_percentile=95,zoom=10,title='SIMCODE (Combined with patterns)')


In [ ]:
fig,ax=plt.subplots(3,2,figsize=(6,10))
gt_ds.renderFigure(axes=ax[0,0],clip_percentile=90,zoom=10,title='Ground truth')
ax[0,1].imshow(mov.mean(0)); ax[0,1].set_title('Measurement average')
# disable axes
ax[0,1].axis('off')
smlm_ds.renderFigure(axes=ax[1,0],zoom=10,clip_percentile=98,title='SMLM')
sf_ds.renderFigure(axes=ax[1,1],clip_percentile=98,zoom=10,title='SIMFLUX')
simcode_nn_ds.renderFigure(axes=ax[2,0],clip_percentile=98,zoom=10,title='SIMCODE NN')
ax[2,1].axis('off'); ax[2,1].set_title('DECODE')
simcode_merged_ds.renderFigure(axes=ax[2,1],clip_percentile=96,zoom=10,title='SIMCODE')
#plt.tight_layout()